In [1]:
# Importar bibliotecas
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import ta
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import GridSearchCV

### Passo 1. Baixar dados históricos entre 12/09/2022 até dia atual


In [2]:
# Ler dados historicos
df = pd.read_csv("acoes_historico_20240912_102107.csv", encoding = 'utf-8')

# Ordenar por data e ticker
df = df.sort_values(['Data','Ticker'])

df.tail()

C:\Users\55119\AppData\Local\Temp\ipykernel_32908\3632272820.py:2: DtypeWarning: Columns (12,13,15,16,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("acoes_historico_20240912_102107.csv", encoding = 'utf-8')


,Ticker,Data,Abertura,MenorValor,MaiorValor,Fechamento,Volume,Dividendos,Desdobramento,Cidade,...,RetornoSobrePatrimonio,FluxoDeCaixaLivre,FluxoDeCaixaOperacional,CrescimentoDosLucros,CrescimentoDaReceita,MargensBrutas,MargensEbitda,MargensOperacionais,MoedaFinanceira,RelacaoPegRetrospectiva
103553,ZAMP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103611,ZAMP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103622,ZAMP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103683,ZAMP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103692,ZAMP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print('*** Dados Brutos ***')
print('')
print(f'Quantidade de linhas: {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

*** Dados Brutos ***

Quantidade de linhas: 177572
Quantidade de colunas: 113


### Passo 2. Limpar dados

In [4]:
df.drop_duplicates(inplace = True)
df.dropna(axis = 0)

print('*** Dados em limpeza ***')
print('')
print(f'Quantidade de linhas: {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

*** Dados em limpeza ***

Quantidade de linhas: 119763
Quantidade de colunas: 113


In [5]:
# Remover linha em que contem NaN na coluna 'Fechamento'
df = df.dropna(subset=['Fechamento'])
print('*** Dados em limpeza ***')
print('')
print(f'Quantidade de linhas: {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

*** Dados em limpeza ***

Quantidade de linhas: 114457
Quantidade de colunas: 113


### Passo 3. Obter dados específicos

In [6]:
tickers = df['Ticker'].drop_duplicates().tolist()
print(f'Total de tickers: {len(tickers)}')
print(tickers)

Total de tickers: 267
['AALR3', 'ABCB4', 'ABEV3', 'AERI3', 'AGRO3', 'AGXY3', 'ALLD3', 'ALUP11', 'ALUP4', 'AMAR3', 'AMBP3', 'AMER3', 'ANIM3', 'ASAI3', 'ATOM3', 'AURE3', 'AZEV3', 'AZEV4', 'AZUL4', 'BBAS3', 'BBDC3', 'BBDC4', 'BEEF3', 'BEES3', 'BIOM3', 'BLAU3', 'BMGB4', 'BMOB3', 'BOBR4', 'BPAC11', 'BRAP3', 'BRBI11', 'BRFS3', 'BRIT3', 'BRKM3', 'BRKM5', 'BRSR3', 'BRSR6', 'CAMB3', 'CAML3', 'CASH3', 'CCRO3', 'CEAB3', 'CEBR6', 'CMIG3', 'CMIG4', 'CMIN3', 'COGN3', 'CPFE3', 'CPLE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSED3', 'CSNA3', 'CSUD3', 'CTSA4', 'CVCB3', 'CXSE3', 'CYRE3', 'DASA3', 'DESK3', 'DEXP3', 'DIRR3', 'DMVF3', 'DOHL4', 'DOTZ3', 'EALT4', 'EGIE3', 'ELET3', 'ELET6', 'ELMD3', 'EMBR3', 'ENEV3', 'ENGI11', 'ENGI3', 'ENJU3', 'EQPA3', 'EQTL3', 'ESPA3', 'ETER3', 'EUCA4', 'EZTC3', 'FESA4', 'FIQE3', 'FLRY3', 'GGBR3', 'GGBR4', 'GGPS3', 'GMAT3', 'GOAU3', 'GOAU4', 'GOLL4', 'GRND3', 'GUAR3', 'HAGA3', 'HAPV3', 'HBOR3', 'HBRE3', 'HYPE3', 'IFCM3', 'IGTI3', 'INEP3', 'INTB3', 'IRBR3', 'ITSA3', 'ITSA4', 'ITUB3', '

### Passo 4. Processar os dados obtidos

##### Os valores para média móvel de 10 e 50 dias, somente estarão disponível após os primeiros 10 e 50 dias respectivamente

#### RSI: Esse método de investimento de alto risco busca obter ganhos através da previsão de oscilações de curto prazo no mercado financeiro.

Fonte: https://www.suno.com.br/artigos/rsi/#:~:text=RSI:%20Conhecendo%20o%20indicador%20Relative%20Strength%20Index

In [7]:
frames = [] # Armazenar dados

for ticker in tickers:
    data = df[df['Ticker'] == ticker].copy()

    # Criar variaveis preditoras
    data['SMA10'] = data['Fechamento'].rolling(window=10).mean().fillna(0.00) # Media movel dos ultimos 10 dias
    data['SMA50'] = data['Fechamento'].rolling(window=50).mean().fillna(0.00) # Media movel dos ultimos 50 dias
    data['VariacaoDiaria'] = data['Fechamento'].pct_change() # Variacao diaria de preco
    data['RSI'] = ta.momentum.RSIIndicator(df['Fechamento'], window=14).rsi() # <= 7 dias = Curto Prazo | > 7 e <= 14 = Medio Prazo | > 14 = Longo Prazo
    data['RSI'] =  data['RSI'].fillna( data['RSI'].mean()) # Altera valores nulos pela media
    data['MME10'] = data['Fechamento'].ewm(span=10).mean()
    data['MME50'] = data['Fechamento'].ewm(span=50).mean()

    # Remover valores nulos apos calculos
    data.dropna(subset = ['VariacaoDiaria'], inplace = True) 

    # Classes de tendencia
    conditions = [
        (data['VariacaoDiaria'] > 1), # Alta
        (data['VariacaoDiaria'] < -1),  # Baixa
        (abs(data['VariacaoDiaria']) <= 1)  # Neutro
    ]

    choices = ['alta','baixa','neutra']
    data['Tendencia'] = np.select(conditions, choices, default = 'semclassificacao') # Atribuir 'classe' para valor da VariacaoDiaria

    frames.append(data)       

# Unir dataframes
union_data = pd.concat(frames) 

In [8]:
# Exibir valores
union_data['MME10'], union_data['MME50']


(145614    19.713000
 145623    19.856512
 145632    20.032272
 145644    20.155061
 145667    20.221278
             ...    
 177565    18.640000
 177566    18.640000
 177569    18.640000
 177570    18.640000
 177571    18.820648
 Name: MME10, Length: 114190, dtype: float64,
 145614    19.718600
 145623    19.840449
 145632    19.986188
 145644    20.088680
 145667    20.147728
             ...    
 177565    18.640000
 177566    18.640000
 177569    18.640000
 177570    18.640000
 177571    18.742287
 Name: MME50, Length: 114190, dtype: float64)

In [9]:
# Verifica se sobrou algum valor nulo (a mesma quantidade de valores que o item 2)

union_data.dropna(subset = ['SMA10','SMA50','VariacaoDiaria','MME10','MME50'], inplace = True)

print(f'Valores não nulos para SMA10: {union_data['SMA10'].count()}')
print(f'Valores não nulos para SMA50: {union_data['SMA50'].count()}')
print(f'Valores não nulos para RSI: {union_data['RSI'].count()}')

Valores não nulos para SMA10: 114190
Valores não nulos para SMA50: 114190
Valores não nulos para RSI: 114190


In [10]:
# Preparar dados para treino
# x = union_data[['SMA10','SMA50','RSI']].values # features
x = union_data[['SMA10','SMA50','MME10','MME50']].values # features
y = union_data['Tendencia'].values # labels

In [12]:
x

array([[ 0.        ,  0.        , 19.713     , 19.7186    ],
       [ 0.        ,  0.        , 19.85651163, 19.84044927],
       [ 0.        ,  0.        , 20.03227228, 19.98618788],
       ...,
       [ 0.        ,  0.        , 18.64      , 18.64      ],
       [ 0.        ,  0.        , 18.64      , 18.64      ],
       [18.726     ,  0.        , 18.8206483 , 18.74228657]])

In [13]:
# Dividir dados em treino
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
# Normalizar dados
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [17]:
# Treinar o modelo
smv_model = SVC(kernel='rbf', class_weight='balanced')
smv_model.fit(x_train, y_train)

SVC(class_weight='balanced')

In [18]:
# Previsoes
y_pred = smv_model.predict(x_test)

In [19]:
# Comparando y_test (coluna Tendencia real) com y_pred (previsões do modelo)
print("Acurácia:", accuracy_score(y_test, y_pred))

# Relatório detalhado de classificação
print(classification_report(y_test, y_pred))

Acurácia: 0.8240213678956125


c:\Users\55119\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\55119\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

        alta       0.00      0.00      0.00         0
      neutra       1.00      0.82      0.90     22838

    accuracy                           0.82     22838
   macro avg       0.50      0.41      0.45     22838
weighted avg       1.00      0.82      0.90     22838



c:\Users\55119\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Passo 5. Verificar proporção da tendência
##### Acurácia do modelo ainda está muito baixa, então vamos tentar balancear as classes, visto que já muito mais dados como neutro

In [ ]:
# Contagem dos valores de cada classe
class_counts = union_data['Tendencia'].value_counts()

# Calcular a porcentagem de  cada classe
class_percentage = (class_counts / len(union_data)) * 100

# Exibir o número de exemplos e a porcentagem de cada classe
print("Distribuição das classes (Alta, Baixa, Neutra):")
print(class_counts)
print("\nPorcentagem das classes:")
print(class_percentage)


##### Balanceamento de classe
Reduzir a quantidade da classe majoritária

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Aplicar RandomUnderSampler para undersampling
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(x, y)

# Verificar a distribuição das classes após o undersampling
print("Distribuição das classes após undersampling:")
print(Counter(y_resampled))


In [17]:
# Dividir os dados em treino e teste
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [175]:
smv_model = SVC(kernel='rbf', class_weight='balanced')
smv_model.fit(x_train, y_train)

# Previsoes
y_pred = smv_model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Comparando y_test (coluna Tendencia real) com y_pred (previsões do modelo)
print("Acurácia:", accuracy_score(y_test, y_pred))

# Relatório detalhado de classificação
print(classification_report(y_test, y_pred))
